The matrix element (weak field)
=============

In [ ]:
from __future__ import print_function
import numpy as np
from sftrident import current2
from sftrident import coordinate
from sftrident import alpha
from sftrident.weakField import fsquare
from sftrident.kinematics import photoNumBW
from sftrident import matrixElement

In [ ]:
def buildArr_lcc(Nss,N1m,N1x,N1y,N2m,N2x,N2y,eps=1e-5):
    ss = np.linspace(3.1,4.1,Nss)
    p1m = np.linspace(eps,1.5,N1m)
    p1x = np.linspace(-2,2,N1x)
    p1y = np.linspace(-2,2,N1y)
    p2m = np.linspace(eps,1.5,N2m)
    p2x = np.linspace(-2,2,N2x)
    p2y = np.linspace(-2,2,N2y)
    return [ss,p1m,p1x,p1y,p2m,p2x,p2y]

mass = 1.0
Narr1 = (2,2,2,2,2,2,2)
frame = 'lab'

arrLCC = buildArr_lcc(*Narr1)
kinObj = coordinate.momenta(*arrLCC,mass = mass,frame = frame,coordSys='lcc')
J0c = current2.J0(kinObj.P3,kinObj.P,kinObj.K,mode='c') # compton part
J0bw = current2.J0(kinObj.P2,kinObj.P1,kinObj.K,mode='bw') # bw part
J1bw = current2.J0(kinObj.P2,kinObj.P1,kinObj.K,mode='bw') # bw part

tempDPHI = 50.0
config = {'a0':1e-4,'mass':1.0,'xi':0.0,'dPhi':tempDPHI,'psBounds':[-tempDPHI,tempDPHI],'envelope':'cos','pulseOpt':['analytic'],'deg':2500,'wf':True,'system':'lab'}
alphaFKTbw = alpha([kinObj.K,kinObj.P2,kinObj.P1],config)
alpha1bw = alphaFKTbw(1)


F = fsquare().getF() #change name to  F!!
Fsq = lambda s : F(s,config['dPhi'])

#k,-p,p1,p2,p3

pNum = photoNumBW(0.0,[kinObj.K,-kinObj.P,kinObj.P1,kinObj.P2,kinObj.P3])

print('pNum[0]',pNum[0])

print("mom size:",kinObj.grid.size)
print("alpha size:",alpha1bw.shape)
print(pNum.shape)


In [ ]:
def prodCurr(curr1,curr2):
    def temp(s11,s12,s21,s22):
        return curr1(0,s11,s12)*curr2(0,s21,s22) - curr1(1,s11,s12)*curr2(1,s21,s22)-curr1(2,s11,s12)*curr2(2,s21,s22)-curr1(3,s11,s12)*curr2(3,s21,s22)
    return temp

def M11(s1,s,s2,s3):
    termBW = lambda mu,xs2,xs3: J1bw(mu,xs2,xs3) - alpha1bw/pNum*J0bw(mu,xs2,xs3)
    tempProd = prodCurr(J0c,termBW)
    #print(tempProd(s1,s,s2,s3))
    return 2.0*np.pi*tempProd(s1,s,s2,s3)*Fsq(pNum)/((kinObj.P - kinObj.P1)*(kinObj.P - kinObj.P1))

In [ ]:
for s1 in (0,1):
    for s in (0,1):
        for s2 in (0,1):
            for s3 in (0,1):
                print("-"*20)
                print(s1,s,s2,s3)
                print(M11(s1,s,s2,s3)[0])

In [ ]:
matObj = matrixElement(config)
LCCpkt = [el[0] for el in arrLCC]
kinPara = [LCCpkt[0],LCCpkt[2],LCCpkt[3],LCCpkt[1], LCCpkt[5],LCCpkt[6],LCCpkt[4]]
matObj.setKin(kinPara)
resOld = matObj.evalMdirPartBW()
print(resOld)

In [ ]:
print('====== C mom')
print('K',kinObj.K[0])
print('P1',kinObj.P3[0])
print('P',kinObj.P[0])
print('====== BW mom')
print('K',kinObj.K[0])
print('P2',kinObj.P2[0])
print('P3',kinObj.P1[0])

In [ ]:
print("U(0)\n\t",J0c.sp2[0][:,0])
print("U(1)\n\t",J0c.sp2[1][:,0])

print("U1(0)\n\t",J0bw.sp2[0][:,0])
print("U1(1)\n\t",J0bw.sp2[1][:,0])

print("U2(0)\n\t",J0bw.sp1[0][:,0])
print("U2(1)\n\t",J0bw.sp1[1][:,0])

print("U3(0)\n\t",J0c.sp1[0][:,0])
print("U3(1)\n\t",J0c.sp1[1][:,0])